In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score,classification_report
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization

In [4]:
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')
X_train = X_train.drop(["Velicina"],axis = 1)
X_test = X_test.drop(["Velicina"],axis = 1)

%store -r X_train_SMOTE
%store -r y_train_SMOTE
X_train_SMOTE = pd.DataFrame(X_train_SMOTE.iloc[:, :-1], columns=X_train_SMOTE.columns[:-1])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

from sklearn.model_selection import train_test_split
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y_train_encoded, test_size=0.15, random_state=42, stratify=y_train_encoded
)

num_classes = len(np.unique(y_train))


c:\Users\andje\anaconda3-1\envs\myenv\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andje\anaconda3-1\envs\myenv\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [23]:
%store -r class_weight
%store -r X_train_SMOTE
%store -r y_train_SMOTE
X_train_SMOTE = pd.DataFrame(X_train_SMOTE.iloc[:, :-1], columns=X_train_SMOTE.columns[:-1])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_SMOTE)
X_test_scaled = scaler.transform(X_test)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_SMOTE)
y_test_encoded = label_encoder.transform(y_test)

from sklearn.model_selection import train_test_split
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y_train_encoded, test_size=0.15, random_state=42, stratify=y_train_encoded
)

num_classes = len(np.unique(y_train))

c:\Users\andje\anaconda3-1\envs\myenv\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andje\anaconda3-1\envs\myenv\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


# Grid search

In [47]:
def create_model(neurons_layer1=10, neurons_layer2=140, num_layers=2, dropout_rate=0.0, optimizer='adam', epochs=100, batch_size=32):
    model = Sequential()
    model.add(Dense(neurons_layer1, input_dim=X_train.shape[1], activation='relu'))
    for _ in range(num_layers - 1):
        model.add(Dense(neurons_layer2, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
    return model

model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {
    'neurons_layer1': [20, 30],
    'neurons_layer2': [64, 140],
    'num_layers': [2, 6],
    'optimizer': ['rmsprop'],
    'dropout_rate': [0.2, 0.4],
    'epochs': [200, 300],
    'batch_size': [32, 64]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=make_scorer(accuracy_score), cv=3, verbose=2)

grid_result = grid.fit(X_train_split, y_train_split)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

best_params = grid_result.best_params_
final_model = create_model(**best_params)

history = final_model.fit(X_train_split, y_train_split, epochs=best_params['epochs'], 
                          batch_size=best_params['batch_size'], validation_data=(X_val_split, y_val_split), 
                          verbose=2)

test_loss, test_accuracy = final_model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


C:\Users\andje\AppData\Local\Temp\ipykernel_25292\370474042.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)


Fitting 3 folds for each of 64 candidates, totalling 192 fits
[CV] END batch_size=32, dropout_rate=0.2, epochs=200, neurons_layer1=20, neurons_layer2=64, num_layers=2, optimizer=rmsprop; total time=   2.3s
[CV] END batch_size=32, dropout_rate=0.2, epochs=200, neurons_layer1=20, neurons_layer2=64, num_layers=2, optimizer=rmsprop; total time=   2.4s
[CV] END batch_size=32, dropout_rate=0.2, epochs=200, neurons_layer1=20, neurons_layer2=64, num_layers=2, optimizer=rmsprop; total time=   2.4s
[CV] END batch_size=32, dropout_rate=0.2, epochs=200, neurons_layer1=20, neurons_layer2=64, num_layers=6, optimizer=rmsprop; total time=   4.1s
[CV] END batch_size=32, dropout_rate=0.2, epochs=200, neurons_layer1=20, neurons_layer2=64, num_layers=6, optimizer=rmsprop; total time=   3.9s
[CV] END batch_size=32, dropout_rate=0.2, epochs=200, neurons_layer1=20, neurons_layer2=64, num_layers=6, optimizer=rmsprop; total time=   4.3s
[CV] END batch_size=32, dropout_rate=0.2, epochs=200, neurons_layer1=20, n

# Prikaz prvih 10 najboljih rezultata

In [49]:
pd.set_option('display.max_colwidth', None)

results_df = pd.DataFrame(grid_result.cv_results_)
sorted_results = results_df.sort_values(by='mean_test_score', ascending=False)

top_10_results = sorted_results.head(10)
print(top_10_results[['params', 'mean_test_score', 'std_test_score']])


                                                                                                                                          params  \
28   {'batch_size': 32, 'dropout_rate': 0.4, 'epochs': 300, 'neurons_layer1': 30, 'neurons_layer2': 64, 'num_layers': 2, 'optimizer': 'rmsprop'}   
11  {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 300, 'neurons_layer1': 20, 'neurons_layer2': 140, 'num_layers': 6, 'optimizer': 'rmsprop'}   
13   {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 300, 'neurons_layer1': 30, 'neurons_layer2': 64, 'num_layers': 6, 'optimizer': 'rmsprop'}   
27  {'batch_size': 32, 'dropout_rate': 0.4, 'epochs': 300, 'neurons_layer1': 20, 'neurons_layer2': 140, 'num_layers': 6, 'optimizer': 'rmsprop'}   
15  {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 300, 'neurons_layer1': 30, 'neurons_layer2': 140, 'num_layers': 6, 'optimizer': 'rmsprop'}   
60   {'batch_size': 64, 'dropout_rate': 0.4, 'epochs': 300, 'neurons_layer1': 30, 'neurons_layer2': 64, 'num_lay

# Hiperparametarsko podešavanje najboljeg modela

In [8]:
def create_model_with_batch_norm(neurons_layer1=30, neurons_layer2=64, num_layers=2, dropout_rate=0.4, optimizer='rmsprop'):
    model = Sequential()
    model.add(Dense(neurons_layer1, input_dim=X_train_scaled.shape[1], activation='relu'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.5))
    for _ in range(num_layers - 1):
        model.add(Dense(neurons_layer2, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
    return model

Za svaku epohu cuvamo model i izvlacimo najbolji

In [ ]:
model = create_model_with_batch_norm()

checkpoint = ModelCheckpoint('modeli/model_epoch_{epoch:02d}.h5', monitor='accuracy', save_best_only=False)

history = model.fit(X_train_scaled, y_train_encoded, epochs=300, 
                    batch_size=32, verbose=2, callbacks=[checkpoint])

In [32]:
final_model_weight = load_model("best_model_weight.h5")
y_pred = final_model_weight.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

class_names = label_encoder.classes_

print(classification_report(y_test_encoded, y_pred_classes, target_names=class_names))

test_loss, test_accuracy = final_model_weight.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
final_model_weight.summary() 

              precision    recall  f1-score   support

           C       0.96      0.85      0.90        82
           K       0.70      0.76      0.73        25
           N       0.77      0.96      0.86        25
           P       0.29      0.50      0.36         4
           R       0.57      0.50      0.53        16

    accuracy                           0.81       152
   macro avg       0.66      0.71      0.68       152
weighted avg       0.83      0.81      0.81       152

5/5 [==============================] - 0s 1ms/step - loss: 1.1754 - accuracy: 0.8092
Test Accuracy: 80.92%
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 30)                1950      
                                                                 
 batch_normalization_8 (Batc  (None, 30)               120       
 hNormalization)                                    

In [17]:
final_model_SMOTE = load_model("best_model_SMOTE.h5")
y_pred = final_model_SMOTE.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

class_names = label_encoder.classes_

print(classification_report(y_test_encoded, y_pred_classes, target_names=class_names))

test_loss, test_accuracy = final_model_SMOTE.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
final_model_SMOTE.summary() 

              precision    recall  f1-score   support

           C       0.94      0.94      0.94        82
           K       0.75      0.60      0.67        25
           N       0.92      0.92      0.92        25
           P       0.33      0.25      0.29         4
           R       0.55      0.75      0.63        16

    accuracy                           0.84       152
   macro avg       0.70      0.69      0.69       152
weighted avg       0.85      0.84      0.84       152

5/5 [==============================] - 0s 1ms/step - loss: 1.0969 - accuracy: 0.8421
Test Accuracy: 84.21%
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 30)                1950      
                                                                 
 batch_normalization_4 (Batc  (None, 30)               120       
 hNormalization)                                    

# Finalni model

In [108]:
final_model = load_model('best_model1.h5')
y_pred = final_model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

class_names = label_encoder.classes_

print(classification_report(y_test_encoded, y_pred_classes, target_names=class_names))

test_loss, test_accuracy = final_model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
final_model.summary() 


              precision    recall  f1-score   support

           C       0.95      0.98      0.96        82
           K       0.82      0.56      0.67        25
           N       0.96      0.96      0.96        25
           P       0.67      0.50      0.57         4
           R       0.52      0.75      0.62        16

    accuracy                           0.87       152
   macro avg       0.78      0.75      0.76       152
weighted avg       0.88      0.87      0.87       152

5/5 [==============================] - 0s 1ms/step - loss: 0.5906 - accuracy: 0.8684
Test Accuracy: 86.84%
Model: "sequential_257"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1327 (Dense)          (None, 30)                1950      
                                                                 
 batch_normalization_158 (Ba  (None, 30)               120       
 tchNormalization)                                